
##Avaliações acadêmicas
A anáise tem como objetivo expor os professores que possuem as piores avaliações acadêmicas nos anos de 2015 ~ 2017. Na pesquisa foram considerados apenas professores ativos do Departamento de Informática e Matemática Aplicada (DIMAp) e do Instituto Metrópole Digital (IMD).

##Observação dos dados
Os dados serão expostos em 3 tabelas e em um gráfico, conforme definido a seguir:
* Tabela 1. Atuacao profissional - contendo o top 5 dos piores professores do ano de 2015;
* Tabela 2. Atuacao profissional - contendo o top 5 dos piores professores do ano de 2016;
* Tabela 3. Atuacao profissional - Contendo o top 10 dos piores professores do ano de 2017;
* Gráfico 1. Atuacao profissional - relação entre nota x departamento x ano.






In [3]:
import pandas as pd
docentes  = pd.read_csv('datasets/docentes/docentes.csv', error_bad_lines=False, delimiter=";")
docentes = docentes.drop(columns=['siape', 'tipo_jornada_trabalho', 'categoria', 'classe_funcional', 'admissao'])
avaliacao_docentes = pd.read_csv('datasets/docentes/avaliacaodocencia.csv', error_bad_lines=False, delimiter=";")

#drop em colunas não interessantes para a análise
avaliacao_docentes = avaliacao_docentes.drop(columns=['id_turma', 'periodo', 'postura_profissional_DP', 'atuacao_profissional_DP', 'autoavaliacao_aluno_media', 'autoavaliacao_aluno_DP'])

#merge nas tabelas de docentes e avaliacoes
docentes_e_avaliacoes = pd.merge(docentes, avaliacao_docentes, how='left', left_on='id_servidor', right_on='id_docente',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

#drop em colunas não interessantes para a análise
docentes_e_avaliacoes = docentes_e_avaliacoes.drop(columns=['id_docente', 'id_unidade_lotacao'])

#selecionando apenas dados de 2015 em diante, ou seja, até 2017
docentes_e_avaliacoes['ano'] = pd.to_numeric(docentes_e_avaliacoes['ano'], errors='coerce')
docentes_e_avaliacoes = docentes_e_avaliacoes[docentes_e_avaliacoes['ano'] > 2014]

#Pegando apenas docentes das seguintes lotações:
#DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA  
#INSTITUTO METROPOLE DIGITAL
docentes_e_avaliacoes = docentes_e_avaliacoes[(docentes_e_avaliacoes['lotacao'] == 'INSTITUTO METROPOLE DIGITAL') | (docentes_e_avaliacoes['lotacao'] == 'DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA')]

#A analise será feita apenas em docentes ativos
docentes_e_avaliacoes = docentes_e_avaliacoes[docentes_e_avaliacoes['vinculo'] == 'Ativo Permanente']


#debug
#docentes_e_avaliacoes[(docentes_e_avaliacoes['id_servidor']==22764) & (docentes_e_avaliacoes['ano']==2017)].mean()

#agrupando os docentes e calculando as medias da quantidade de discentes e as avaliações
docentes_aval_imd_dimap = docentes_e_avaliacoes.groupby(['id_servidor', 'ano']).mean().reset_index()

#merge na tabela anterior para reunir o restante das informações necessárioas
docentes_aval_imd_dimap = pd.merge(docentes_aval_imd_dimap, docentes_e_avaliacoes[['nome','formacao','lotacao', 'id_servidor']],  left_on='id_servidor', right_on='id_servidor')

#removendo valores duplicados
docentes_aval_imd_dimap = docentes_aval_imd_dimap.drop_duplicates()
docentes_aval_imd_dimap = docentes_aval_imd_dimap.drop(columns='postura_profissional_media')

In [4]:
#Separando informações para organizar a visualização
#Pegando os dados de cada ano. P.S. Lembrar de ordenar pelo tipo da avaliacao quando for extrair a tabela
aval_dimap_imd_2015 = docentes_aval_imd_dimap[docentes_aval_imd_dimap['ano'] == 2015]
aval_dimap_imd_2016 = docentes_aval_imd_dimap[docentes_aval_imd_dimap['ano'] == 2016]
aval_dimap_imd_2017 = docentes_aval_imd_dimap[docentes_aval_imd_dimap['ano'] == 2017]

#Pegando media geral por lotacao e por ano
imd_dimap_geral = docentes_e_avaliacoes.groupby(['lotacao', 'ano']).mean().reset_index()
imd_dimap_geral = imd_dimap_geral.drop(columns=['id_servidor', 'postura_profissional_media'])
#imd_dimap_geral.head()

In [5]:
#Observação dos dados
aval_dimap_imd_2015_atuacao = aval_dimap_imd_2015.sort_values(by='atuacao_profissional_media', ascending=True)
aval_dimap_imd_2015_atuacao.head(10)

,id_servidor,ano,qtd_discentes,atuacao_profissional_media,nome,formacao,lotacao
999,5756871,2015.0,8.000000,6.745000,EDUARDO NOGUEIRA CUNHA,MESTRADO,INSTITUTO METROPOLE DIGITAL
1218,5757768,2015.0,43.000000,6.990000,UMBERTO RIVIECCIO,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
325,5752241,2015.0,24.666667,7.086667,EDUARDO HENRIQUE DA SILVA ARANHA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
176,26012,2015.0,17.666667,7.106667,MARCIA JACYNTHA NUNES RODRIGUES LUCENA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
399,5754320,2015.0,19.000000,7.340000,LEONARDO CUNHA DE MIRANDA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
307,5751871,2015.0,26.500000,7.740000,UIRA KULESZA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
69,24798,2015.0,22.666667,8.386667,REGIVAN HUGO NUNES SANTIAGO,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
745,5755963,2015.0,18.000000,8.532500,WELLINGTON SILVA DE SOUZA,MESTRADO,INSTITUTO METROPOLE DIGITAL
872,5756592,2015.0,11.666667,8.570000,ANTONIO CARLOS GAY THOME,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
200,26044,2015.0,30.666667,8.583333,BENJAMIN RENE CALLEJAS BEDREGAL,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA


In [6]:
#Observação dos dados
aval_dimap_imd_2016_atuacao = aval_dimap_imd_2016.sort_values(by='atuacao_profissional_media', ascending=True)
aval_dimap_imd_2016_atuacao.head(10)

,id_servidor,ano,qtd_discentes,atuacao_profissional_media,nome,formacao,lotacao
184,26012,2016.0,18.333333,6.933333,MARCIA JACYNTHA NUNES RODRIGUES LUCENA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
76,24798,2016.0,36.500000,7.300000,REGIVAN HUGO NUNES SANTIAGO,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
387,5753044,2016.0,16.000000,7.743333,LYRENE FERNANDES DA SILVA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
97,24853,2016.0,24.333333,7.890000,ANNE MAGALY DE PAULA CANUTO,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
313,5751871,2016.0,26.000000,7.975000,UIRA KULESZA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
594,5755408,2016.0,19.000000,8.110000,AUGUSTO JOSE VENANCIO NETO,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1279,5757958,2016.0,29.500000,8.242500,ATHANASIOS TSOUANAS,DOUTORADO,INSTITUTO METROPOLE DIGITAL
206,26044,2016.0,36.000000,8.290000,BENJAMIN RENE CALLEJAS BEDREGAL,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
123,24949,2016.0,21.000000,8.480000,ANDRE MAURICIO CUNHA CAMPOS,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
995,5756850,2016.0,16.000000,8.540000,ISAAC FRANCO FERNANDES,MESTRADO,INSTITUTO METROPOLE DIGITAL


In [7]:
#Observação dos dados
aval_dimap_imd_2017_atuacao = aval_dimap_imd_2017.sort_values(by='atuacao_profissional_media', ascending=True)
aval_dimap_imd_2017_atuacao.head(10)

,id_servidor,ano,qtd_discentes,atuacao_profissional_media,nome,formacao,lotacao
83,24798,2017.0,16.0,7.980000,REGIVAN HUGO NUNES SANTIAGO,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
212,26044,2017.0,49.0,8.390000,BENJAMIN RENE CALLEJAS BEDREGAL,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1286,5757958,2017.0,30.0,8.423333,ATHANASIOS TSOUANAS,DOUTORADO,INSTITUTO METROPOLE DIGITAL
192,26012,2017.0,20.0,8.555000,MARCIA JACYNTHA NUNES RODRIGUES LUCENA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
392,5753044,2017.0,22.0,8.610000,LYRENE FERNANDES DA SILVA,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1232,5757768,2017.0,15.0,8.635000,UMBERTO RIVIECCIO,DOUTORADO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
957,5756813,2017.0,39.5,8.690000,RAFAELA HORACINA SILVA ROCHA SOARES,MESTRADO,INSTITUTO METROPOLE DIGITAL
476,5755108,2017.0,25.5,8.830000,CARLOS EDUARDO DA SILVA,DOUTORADO,INSTITUTO METROPOLE DIGITAL
677,5755872,2017.0,33.0,8.883333,SAMYR SILVA BEZERRA JACOME,DOUTORADO,INSTITUTO METROPOLE DIGITAL
1249,5757844,2017.0,32.5,8.922500,SILVIO COSTA SAMPAIO,DOUTORADO,INSTITUTO METROPOLE DIGITAL


In [8]:
imd_dimap_geral

,lotacao,ano,qtd_discentes,atuacao_profissional_media
0,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA,2015.0,20.602410,8.886145
1,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA,2016.0,23.186047,8.819767
2,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA,2017.0,21.100000,9.189800
3,INSTITUTO METROPOLE DIGITAL,2015.0,25.120370,9.117407
4,INSTITUTO METROPOLE DIGITAL,2016.0,28.888889,9.198462
5,INSTITUTO METROPOLE DIGITAL,2017.0,34.076923,9.381846


In [9]:
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge

#Pegando os dados de cada lotação por ano
dimap_onlyaval_by_year = list(imd_dimap_geral.loc[imd_dimap_geral['lotacao']=='DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA', 'atuacao_profissional_media'])
imd_onlyaval_by_year = list(imd_dimap_geral.loc[imd_dimap_geral['lotacao']=='INSTITUTO METROPOLE DIGITAL', 'atuacao_profissional_media'])

output_file("avaliacao_atuacao_profissional.html")

years = ['2015', '2016', '2017']
location = ['DIMAp', 'IMD']


data = {'years' : years,
        'DIMAp' : dimap_onlyaval_by_year,
        'IMD'   : imd_onlyaval_by_year}

source = ColumnDataSource(data=data)

p = figure(x_range=years, y_range=(0, 10), plot_height=250, title=" ",
           toolbar_location=None, tools="")

p.vbar(x=dodge('years',  -0.1,  range=p.x_range), top='DIMAp', width=0.15, source=source,
       color="#48D1CC", legend=value("DIMAp"))

p.vbar(x=dodge('years',  0.1, range=p.x_range), top='IMD', width=0.15, source=source,
       color="#CD5C5C", legend=value("IMD"))

p.x_range.range_padding = 0.5
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "vertical"

show(p)